In [1]:
import logging

logger = logging.getLogger("nlstruct")
logger.setLevel(logging.DEBUG)


In [2]:
import string
import logging
import pandas as pd

import sys
sys.path.insert(0,'/home/ytaille/deep_multilingual_normalization')

from nlstruct.utils import torch_clone
from nlstruct.utils import torch_global as tg
from deep_multilingual_normalization.preprocess import preprocess, load_quaero
from deep_multilingual_normalization.train import train_step1, train_step2, clear
from deep_multilingual_normalization.eval import compute_scores, predict

from transformers import AutoModel
from deep_multilingual_normalization.model import Classifier, FastClusteredIPSearch

from notebook_utils import *

In [3]:
from deep_multilingual_normalization.create_classifiers import create_classifiers

classifier, classifier2 = create_classifiers()

Available CUDA devices: 1
Current device: cuda:0
Using cache /home/ytaille/data/cache/preprocess_training_data/b5e7f1ec57d0ed68
Loading /home/ytaille/data/cache/preprocess_training_data/b5e7f1ec57d0ed68/output.pkl... 
Loading MRCONSO...
Deduplicating MRCONSO...
French synonyms: 174317
French labels: 84172
Quaero mentions: 5714
Mirrored labels: 84845
Queried english labels: 84845
Total deduplicated synonyms: 781723
Total deduplicated labels: 84845
Lock 139875591135304 acquired on /home/ytaille/.cache/torch/transformers/33b56ce0f312e47e4d77a57791a4fc6233ae4a560dd2bdd186107058294e58ab.fcb1786f49c279f0e0f158c9972b9bd9f6c0edb5d893dcb9b530d714d86f0edc.lock
Lock 139875591135304 released on /home/ytaille/.cache/torch/transformers/33b56ce0f312e47e4d77a57791a4fc6233ae4a560dd2bdd186107058294e58ab.fcb1786f49c279f0e0f158c9972b9bd9f6c0edb5d893dcb9b530d714d86f0edc.lock
Lock 139874431107928 acquired on /home/ytaille/.cache/torch/transformers/bb773818882b0524dc53a1b31a2cc95bc489f000e7e19773ba07846011a6

In [4]:
bert_name = "bert-base-multilingual-uncased"

dataset = load_from_brat("/home/ytaille/data/tmp/ws_inputs")

docs, sentences, tokens, deltas, vocs = preprocess(
    dataset=dataset,
    max_sentence_length=120,
    bert_name=bert_name,
    vocabularies=None,
)

prep = Dataset(
    sentences=sentences,
    tokens=tokens,
    deltas=deltas,
)

batcher, encoded, ids = make_batcher(docs, sentences, tokens)

Dataset: Dataset(
  (docs): 1 * ('doc_id', 'text', 'split')
)
Transform texts... done
Splitting into sentences... Tokenizing... 

/home/ytaille/.conda/envs/deep_multilingual_normalization/lib/python3.6/site-packages/nlstruct/text/chunking/regex.py:75: FutureWarning: split() requires a non-empty pattern match.
  for i, part in enumerate(reg_split.split(txt)):


done
Computing vocabularies...
Will train vocabulary for token
Discovered existing vocabulary (105879 entities) for token
Normalized split, with given vocabulary and no unk
Normalized split, with given vocabulary and no unk
Normalized split, with given vocabulary and no unk


/home/ytaille/.conda/envs/deep_multilingual_normalization/lib/python3.6/site-packages/nlstruct/text/chunking/huggingface.py:11: FutureWarning: doc_id_col is not used anymore in the huggingface_tokenize function
  warnings.warn("doc_id_col is not used anymore in the huggingface_tokenize function", FutureWarning)


done


In [5]:
bert_name = "bert-base-multilingual-uncased"

dataset = load_from_brat("/home/ytaille/data/tmp/ws_inputs")

docs, sentences, tokens, deltas, vocs = preprocess(
    dataset=dataset,
    max_sentence_length=120,
    bert_name=bert_name,
    vocabularies=None,
)

prep = Dataset(
    sentences=sentences,
    tokens=tokens,
    deltas=deltas,
)

batcher, encoded, ids = make_batcher(docs, sentences, tokens)

batch_size = 32
with_tqdm = True
with_groups = False
topk = 1
save_embeds = False

tg.set_device('cuda:0')
device = tg.device

with evaluating(classifier):  # eval mode: no dropout, frozen batch norm, etc
    with torch.no_grad():  # no gradients -> faster
        with tqdm(batcher.dataloader(batch_size=batch_size, device=device, sparse_sort_on="token_mask"), disable=not with_tqdm, mininterval=10.) as bar:
            for batch in bar:
                res = classifier2.forward(
                    tokens=batch["token"],
                    mask=batch["token_mask"],
                    groups=batch["group"] if with_groups else None,
                    return_scores=topk > 0.1,
                    return_embeds=save_embeds,
                )

Dataset: Dataset(
  (docs): 1 * ('doc_id', 'text', 'split')
)
Transform texts... done
Splitting into sentences... Tokenizing... 

/home/ytaille/.conda/envs/deep_multilingual_normalization/lib/python3.6/site-packages/nlstruct/text/chunking/regex.py:75: FutureWarning: split() requires a non-empty pattern match.
  for i, part in enumerate(reg_split.split(txt)):


done
Computing vocabularies...
Will train vocabulary for token
Discovered existing vocabulary (105879 entities) for token
Normalized split, with given vocabulary and no unk
Normalized split, with given vocabulary and no unk
Normalized split, with given vocabulary and no unk


/home/ytaille/.conda/envs/deep_multilingual_normalization/lib/python3.6/site-packages/nlstruct/text/chunking/huggingface.py:11: FutureWarning: doc_id_col is not used anymore in the huggingface_tokenize function
  warnings.warn("doc_id_col is not used anymore in the huggingface_tokenize function", FutureWarning)


done
Available CUDA devices: 1
Current device: cuda:0


100%|██████████| 1/1 [00:00<00:00, 28.50it/s]


In [6]:
best_probs = res['scores'].topk(dim=-1, k=topk)

In [7]:
best_probs

torch.return_types.topk(
values=tensor([[8.3149],
        [9.0723],
        [7.5563],
        [8.5602]], device='cuda:0'),
indices=tensor([[528324],
        [488454],
        [ 98788],
        [192771]], device='cuda:0'))

In [8]:
best_probs.values

tensor([[8.3149],
        [9.0723],
        [7.5563],
        [8.5602]], device='cuda:0')